In [2]:
import torch
import torch.nn as nn

import pandas as pd
import numpy as np
import time

In [3]:
class FareForecastingModel(nn.Module):
    def __init__(self, num_locations, embedding_dim, num_numeric_features, lstm_hidden_dim, lstm_layers):
        super(FareForecastingModel, self).__init__()
        
        # Embedding layers for pick-up and drop-off
        self.pickup_embedding = nn.Embedding(num_locations, embedding_dim)
        self.dropoff_embedding = nn.Embedding(num_locations, embedding_dim)
        
        # LSTM for modeling temporal sequence
        # Suppose our input for LSTM is the concatenation of embeddings + numeric features at each time step
        # The input dimension for LSTM: 2*embedding_dim + num_numeric_features
        lstm_input_dim = 2 * embedding_dim + num_numeric_features
        self.lstm = nn.LSTM(input_size=lstm_input_dim, hidden_size=lstm_hidden_dim, 
                            num_layers=lstm_layers, batch_first=True)
        
        # Fully connected layer to produce the forecast
        self.fc = nn.Linear(lstm_hidden_dim, 1)
        
    def forward(self, pickup_ids, dropoff_ids, numeric_seq):
        # pickup_ids, dropoff_ids are assumed to have shape (batch_size, seq_length)
        # numeric_seq has shape (batch_size, seq_length, num_numeric_features)
        
        # Get embeddings (result shape: (batch_size, seq_length, embedding_dim))
        pickup_emb = self.pickup_embedding(pickup_ids)
        dropoff_emb = self.dropoff_embedding(dropoff_ids)
        
        # Concatenate embeddings with numeric features along last dimension
        # New shape: (batch_size, seq_length, 2*embedding_dim + num_numeric_features)
        lstm_input = torch.cat((pickup_emb, dropoff_emb, numeric_seq), dim=-1)
        
        # Pass through LSTM
        lstm_out, _ = self.lstm(lstm_input)
        # For simplicity, predict using the output at the final time step
        final_output = lstm_out[:, -1, :]
        
        # Forecast output
        forecast = self.fc(final_output)
        return forecast

In [4]:
# Encoding Functions

def circular_encoder(df):
    # Encode dow and hour as sin and cos
    
    df['dow_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7.0)
    df['dow_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7.0)
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24.0)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24.0)
    df.drop(['day_of_week', 'hour'], axis=1, inplace=True)
    
    return df

In [ ]:
def total_amount_base_model(X_train, X_test, y_train, y_test):
    unique_ids = sorted(set(X_train['PULocationID'].unique()).union(set(X_train['DOLocationID'].unique())))
    id_to_index = {loc_id: idx for idx, loc_id in enumerate(unique_ids)}
    
    X_train['PULocationID'] = X_train['PULocationID'].map(id_to_index)
    X_train['DOLocationID'] = X_train['DOLocationID'].map(id_to_index)
    X_test['PULocationID']  = X_test['PULocationID'].map(id_to_index)
    X_test['DOLocationID']  = X_test['DOLocationID'].map(id_to_index)
    
    num_locations = len(unique_ids)

    model = FareForecastingModel(
        num_locations=num_locations,  # Example: number of unique locations
        embedding_dim=8,
        num_numeric_features=X_train.shape[1] - 2,  # Exclude pickup and dropoff IDs
        lstm_hidden_dim=64,
        lstm_layers=2
    )
    start = time.time()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.MSELoss()
    num_epochs = 10
    batch_size = 32
    num_batches = len(X_train) // batch_size
    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        model.train()
        for i in range(num_batches):
            batch_X = X_train[i * batch_size:(i + 1) * batch_size]
            batch_y = y_train[i * batch_size:(i + 1) * batch_size]

            # print(batch_X.shape)

            # Convert to PyTorch tensors
            pickup_ids = torch.tensor(batch_X['PULocationID'].values).long().unsqueeze(1)
            dropoff_ids = torch.tensor(batch_X['DOLocationID'].values).long().unsqueeze(1)
            numeric_features = torch.tensor(batch_X.drop(['PULocationID', 'DOLocationID'], axis=1).values).float().unsqueeze(1)

            batch_y_tensor = torch.tensor(batch_y.values).float()

            # Forward pass
            optimizer.zero_grad()
            outputs = model(pickup_ids, dropoff_ids, numeric_features)
            
            # Compute loss
            loss = criterion(outputs.squeeze(), batch_y_tensor)
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()

    end = time.time()
    print(f"Training Time: {end - start} seconds")

    # Evaluation
    model.eval()
    with torch.no_grad():
        pickup_ids = torch.tensor(X_test['PULocationID'].values).long().unsqueeze(1)
        dropoff_ids = torch.tensor(X_test['DOLocationID'].values).long().unsqueeze(1)
        numeric_features = torch.tensor(X_test.drop(['PULocationID', 'DOLocationID'], axis=1).values).float().unsqueeze(1)

        predictions = model(pickup_ids, dropoff_ids, numeric_features)
        predictions = predictions.squeeze().numpy()

    # Calculate RMSE
    rmse = np.sqrt(np.mean((predictions - y_test.values) ** 2))
    print(f"RMSE: {rmse}")
    return model, rmse

In [6]:
# Loading the data
df_train_base = pd.read_csv('data/train.csv')
df_test_base = pd.read_csv('data/test.csv')

df_train = df_train_base.copy()
df_test = df_test_base.copy()

df_train = circular_encoder(df_train)
df_test = circular_encoder(df_test)

X_train= df_train.drop(['travel_time', 'total_amount'], axis=1)
y_train = df_train['total_amount']

X_test = df_test.drop(['travel_time', 'total_amount'], axis=1)
y_test = df_test['total_amount']

In [19]:
min(df_test_base['PULocationID'].unique())

np.int64(1)

In [20]:
model, rmse = total_amount_base_model(X_train, X_test, y_train, y_test)

Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10
Training Time: 10848.42094707489 seconds


KeyError: "['PULocationID', 'DOLocationID'] not found in axis"

In [ ]:
# Example instantiation:
num_locations = 265
embedding_dim = 8
num_numeric_features = 5
lstm_hidden_dim = 64
lstm_layers = 2

model = FareForecastingModel(num_locations, embedding_dim, num_numeric_features, lstm_hidden_dim, lstm_layers)
print(model)